In [4]:



# Descomentar el git para cargar el archivo train.csv que se encuentra en el repositorio
!tar -xf "/content/drive/My Drive/Copia de noticias50k.tar.xz" 

time: 1.1 s


Cargo las bibliotecas que se utilizaran mas adelante

In [46]:
import numpy as np
import re

import os
from keras.models import Model
from imblearn.over_sampling import SMOTE, RandomOverSampler
# check version number
import imblearn
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import to_categorical
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer 
import nltk.corpus
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import Dense, Input, Flatten, merge, LSTM, Lambda, Dropout, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
!pip install ipython-autotime

%load_ext autotime
nltk.download('stopwords')
stop = stopwords.words('english')
ps = PorterStemmer()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
time: 3.54 s


Algunos comandos de bash para descomprimir los archivos que tengo alojados en mi drive

In [12]:

#!cp glove.zip "/content/drive/My Drive/glove.zip" 
#!unzip "/content/drive/My Drive/glove.zip"
#!tar -xf "/content/drive/My Drive/Copia de noticias50k.tar.xz" 
train = pd.read_json('noticias50k.json')

time: 565 ms


Cargo el archivo de texto que representa las matrices embebidas del modelo pre-entrenado de  glove. Ademas de que creo un diccionario para mapear cada palabra al vector de caracteristicas que el modelo pre-entrenado de GLove otorga.

In [13]:
# Loading the glove embeding vectors from a file
embeddings_index = {}
files = open(os.path.join("./", 'glove.840B.300d.txt'))

time: 1.96 ms


In [ ]:
for line in files:
    
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except ValueError:
        print(line)
        pass
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [16]:
train = pd.read_json('noticias50k.json')

time: 329 ms


In [15]:
print('Found %s word vectors.' % len(embeddings_index))

Found 2195885 word vectors.
time: 1.26 ms


In [17]:
train.columns

Index(['category', 'headline', 'authors', 'link', 'short_description', 'date'], dtype='object')

time: 3.87 ms


Tome las categorias ['CRIME', 'ARTS & CULTURE', 'RELIGION', 'WEIRD NEWS', 'BUSINESS'] Aunque anteriorment habia tomado la categoria de politica y iba quedar demasiado desbalanceado ya que en promedio (exceptuando politics) presentaba 600 muestras y Politics registraba 17000 muestras

In [89]:

categories = ['CRIME', 'ARTS & CULTURE', 'RELIGION', 'WEIRD NEWS', 'BUSINESS']

def clean(comment):

    comment = comment.lower()
    comment = re.sub(r"\n", " ", comment)
    re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", comment)
    re.sub(r"\d+", "", comment)
    output = ""
    first = True
    
    for element in comment.split():

        if element not in stop and len(element) > 1 and element in embeddings_index:
            
            output += element + " " if first else " " + element

            first = False

    return output


X = []
y = []


maximo = 0
for i in train.index:
    
    category = train.loc[i]['category']
    if category in categories:
        temp = train.loc[i]['short_description']
        temp = clean(temp)
        maximo = max(len(temp),maximo)
        """
        for word in train.loc[i]['short_description'].split():

            if word in embeddings_index:
                temp.append(embeddings_index[word])
        """

        X.append(temp)
        y.append(categories.index(category))

X = np.array(X,dtype='<U400')
y = np.array(y)


time: 15.9 s


In [108]:

y_train = to_categorical(np.asarray(y))

time: 2.34 ms


In [147]:
print(np.unique(train['category'], return_counts=True))
print(np.unique(y, return_counts=True))

(array(['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE',
       'COMEDY', 'CRIME', 'EDUCATION', 'ENTERTAINMENT', 'FIFTY',
       'GOOD NEWS', 'GREEN', 'HEALTHY LIVING', 'IMPACT', 'LATINO VOICES',
       'MEDIA', 'PARENTS', 'POLITICS', 'QUEER VOICES', 'RELIGION',
       'SCIENCE', 'SPORTS', 'STYLE', 'TASTE', 'TECH', 'THE WORLDPOST',
       'TRAVEL', 'WEIRD NEWS', 'WOMEN', 'WORLD NEWS', 'WORLDPOST'],
      dtype=object), array([   25,   806,  1723,   726,    83,  2075,   885,   310,  6199,
          39,   156,   771,  1939,   537,   523,  1100,  1602, 17519,
        2318,   579,   215,  1033,   706,   692,   268,  2026,   473,
         946,  1459,  2177,    90]))
(array([0, 1, 2, 3, 4]), array([885, 806, 579, 946, 726]))
time: 74 ms


In [148]:
X.shape

(3942,)

time: 3.96 ms


Proceso de tokenizacion para representar la descripcion de las noticias

In [149]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=maximo)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

time: 138 ms


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Aplicar

In [150]:
print(f"Number of uniques words is: {len(tokenizer.word_index)}")


Number of uniques words is: 7678
time: 2.53 ms


Convierte la esquencia generada por el tokenizador en una matriz de dimension de (3942, 193) siendo el primer elemento la cantidad de muestra y la segunda el numero de caracteristicas


In [151]:
X_train = pad_sequences(sequences, maxlen=700)
X_train.shape

(3942, 700)

time: 42 ms


En esta seccion aplico el oversampleo SMOTE para manejar el desbalanceo de muestras

In [152]:

sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

#randomSampler = RandomOverSampler(random_state=12)

#X_train_res, y_train_res = randomSampler.fit_resample(X_train, y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

time: 1.28 s


In [153]:
print(np.unique(y_train_res, return_counts=True))

(array([0, 1]), array([18920,  4730]))
time: 8.08 ms


In [154]:
print(X_train_res.shape)
print(y_train_res.shape)

(4730, 700)
(4730, 5)
time: 3.43 ms


Creacion de la matriz embebida

In [155]:
print(len(tokenizer.word_index))
print(X_train_res.shape)

print(X.shape)
rows = min(700, len(tokenizer.word_index) + 1)
embedded_matrix =  np.zeros((rows, 300))

for word, value in tokenizer.word_index.items():

    if value < 700  and word in embeddings_index:

        embedded_matrix[value] = embeddings_index[word]

    

7678
(4730, 700)
(3942,)
time: 15.8 ms


In [156]:
embbedded_result = Embedding(700, 
    300, 
    embeddings_initializer=Constant(embedded_matrix),
    input_length=700)
input_ = Input(shape=(700,), dtype='int32')

time: 6.45 ms


Separacion de los datos de entrenamiento, pruebas y validacion

In [157]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_res, y_train_res, test_size=0.3, random_state=42)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

# para asegurar que los los tamanos son correctos
assert X_train.shape[0] ==  Y_train.shape[0]
assert X_test.shape[0] == Y_test.shape[0] 
assert X_valid.shape[0] == Y_valid.shape[0]

time: 21.3 ms


In [158]:
type(embbedded_result)

keras.layers.embeddings.Embedding

time: 4.48 ms


In [159]:
temp = embbedded_result(input_)

input_layer = Conv1D(1024, 5, activation='relu')(temp)

hidden_layer = MaxPooling1D(5)(input_layer)
hidden_layer = Conv1D(512, 5, activation='relu')(hidden_layer)
hidden_layer = MaxPooling1D(5)(hidden_layer)
hidden_layer = Conv1D(256, 5, activation='relu')(hidden_layer)
hidden_layer = GlobalMaxPooling1D()(hidden_layer)
hidden_layer = Dense(128, activation='relu')(hidden_layer)
output_layer = Dense(5, activation='softmax')(hidden_layer)
model = Model(input_, output_layer)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])



time: 130 ms


In [160]:
y_train.shape == y_train.shape

True

time: 3.82 ms


In [161]:
model.fit(X_train, Y_train,batch_size=64, epochs=20, validation_data=(X_valid, Y_valid))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3311 samples, validate on 710 samples
Epoch 1/20
3311/3311 [==============================] - 16s 5ms/step - loss: 1.5248 - acc: 0.3150 - val_loss: 1.4511 - val_acc: 0.3521
Epoch 2/20
3311/3311 [==============================] - 14s 4ms/step - loss: 1.4395 - acc: 0.3651 - val_loss: 1.4784 - val_acc: 0.3296
Epoch 3/20
3311/3311 [==============================] - 14s 4ms/step - loss: 1.3828 - acc: 0.3990 - val_loss: 1.5684 - val_acc: 0.3437
Epoch 4/20
3311/3311 [==============================] - 14s 4ms/step - loss: 1.3273 - acc: 0.4138 - val_loss: 1.6075 - val_acc: 0.3451
Epoch 5/20
3311/3311 [==============================] - 14s 4ms/step - loss: 1.2781 - acc: 0.4391 - val_loss: 1.5555 - val_acc: 0.3099
Epoch 6/20
3311/3311 [==============================] - 14s 4ms/step - loss: 1.2345 - acc: 0.4536 - val_loss: 1.7035 - val_acc: 0.3268
Epoch 7/20
3311/3311 [==============================] - 14s 4ms/step - loss: 1.2006 - acc: 0.4612 - val_loss: 1.7647 - val_acc: 0.3408
Epoch 8/

time: 4min 43s


In [163]:
y_pred = model.predict(X_test)

time: 1.75 s


In [168]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_pred)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

time: 11.2 ms


In [171]:


for i in range(26):
    cm = confusion_matrix(Y_test[:, i], y_pred[:, i] > 0.5)
    if len(cm) == 1:
        print(f"{enc.categories_[0][i]}\nAcurracy: 1\nPrecision: 1\n")
        continue
    
    total = sum(cm[0]) + sum(cm[1])
    answered = cm[0][0] + cm[1][1]
    precision = cm[1][0] / (cm[0][0] + cm[1][0])
    print(f"{enc.categories_[0][i]}\nAcurracy: {answered/total}\nPrecision: {answered/total}\n{cm}\n")
    

0.0
Acurracy: 0.8321579689703809
Precision: 0.8321579689703809
[[556  26]
 [ 93  34]]

1.8598453050229377e-33
Acurracy: 0.7926657263751763
Precision: 0.7926657263751763
[[546  23]
 [124  16]]

1.0780717492296003e-23
Acurracy: 0.8025387870239774
Precision: 0.8025387870239774
[[534  17]
 [123  35]]

4.887764680018665e-20
Acurracy: 0.767277856135402
Precision: 0.767277856135402
[[527  35]
 [130  17]]

1.7337169005689066e-18
Acurracy: 0.771509167842031
Precision: 0.771509167842031
[[526  46]
 [116  21]]



IndexError: ignored

time: 67.1 ms


#Analisis de los resultados obtenidos

Los resultados obtenidos fueron muy malos. El modelo no llego a converger a una solucion adecuada. No se si fue porque tome categorias de noticias que utilizan palabras muy similares o si al crear mi matriz embebida me haya faltado algun parametro. Aunque tambien podria ser el hecho de la version de Glove que tome que es la de 2gb ademas de que utilice el SMOTE para el oversampleo aunque de por si la clases que estaban levemente desbalanceados.  